# Cats and dogs with pretrained convnet (II - data augmentation)

In [1]:
from keras import models
from keras import layers

from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

from keras.applications import VGG16

import os

Using Theano backend.


# Define folders

In [2]:
base_dir = './cats_and_dogs_small/'

In [3]:
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Load pretrained convnet

In [4]:
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

/datalinux/giudice/software/anaconda/anaconda3/envs/deeplearning_theano/lib/python3.7/site-packages/keras/utils/conv_utils.py:82: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])


# Define total model

In [5]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [7]:
print('This is the number of trainable weight tensor before freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weight tensor before freezing the conv base: 30


In [8]:
#model.trainable_weights

In [9]:
conv_base.trainable = False

In [10]:
print('This is the number of trainable weight tensors after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weight tensors after freezing the conv base: 4


In [11]:
model.trainable_weights

[dense_1/kernel, dense_1/bias, dense_2/kernel, dense_2/bias]

# Data augmentation configuration

In [12]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

In [13]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=20, class_mode='binary')

Found 2000 images belonging to 2 classes.


In [15]:
validation_generator = test_datagen.flow_from_directory(validation_dir, target_size=(150, 150), batch_size=20, 
                                                        class_mode='binary')

Found 1000 images belonging to 2 classes.


# Compile

In [16]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'])

# Fit

In [ ]:
#Note: steps_per_epoch=100 so 2000/20=100 where 20 is the dimension of the batch and 2000 is the total train images
#NOT valid anymore because noe there are many more figures, not just 2000!!!!

In [21]:
# history = model.fit_generator(train_generator, steps_per_epoch=100, epochs=10, validation_data=validation_generator, 
#                               validation_steps=50)

history = model.fit_generator(train_generator, steps_per_epoch=2, epochs=3, validation_data=validation_generator, 
                              validation_steps=5)


Epoch 1/3
2/2 [==============================] - 1346s 673s/step - loss: 0.6420 - acc: 0.6500 - val_loss: 0.5992 - val_acc: 0.7170
Epoch 2/3
1/2 [==============>...............] - ETA: 26s - loss: 0.7869 - acc: 0.4000

KeyboardInterrupt: 

# Plots

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

In [ ]:
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()